# Mutation mapping solution

In [29]:
import re
import os
# This whole script has become spagetthi code...
ref_cds_entry = "brca1.txt"
ref_rna_entry = "NM_007294_4.fasta"
ref_protein_entry = "NP_009225_1.fasta"

codon_table = dict(zip([a + b + c for a in "TCAG" for b in "TCAG" for c in "TCAG"], 
                       'FFLLSSSSYY**CC*WLLLLPPPPHHQQRRRRIIIMTTTTNNKKSSRRVVVVAAAADDEEGGGG'))
test_seq = "AAACCATGCCCCCCGGGAAATAACCC"

def find_orfs(seq):
    stops = "TAA TAG TGA".split()
    len_seq = len(seq)
    start_pos = [m.start() for m in re.finditer('ATG', seq)]
    start_ends = []
    for i in start_pos:
        start_codon_pos = i
        start = start_codon_pos
        max_codons = (len_seq - start_codon_pos) // 3
        for codon in range(max_codons):
            codon_seq = seq[start: start + 3]
            start += 3
            if codon_seq in stops:
                stop_codon_pos = start
                break
        pos = (start_codon_pos + 1, stop_codon_pos)
        start_ends.append(pos)
    return start_ends
    
def compare_rna(ref, seq):
    for pos, base in enumerate(ref):
        if base != seq[pos]:
            break
    return base, seq[pos], pos

def compare_prot(ref, seq):
    for pos, aa in enumerate(ref):
        if aa != seq[pos]:
            break
    if aa == "*" and seq[pos] == "*":
        return "silent", "silent", None
    else:
        return aa, seq[pos], pos + 1

def map_to_refseq(diff_pos, orf_pos_ref_rna):
    start_orf = orf_pos_ref_rna[0]
    diff_pos = diff_pos[2]
    pos = diff_pos + start_orf
    return pos
    
def translate(seq, start, stop, codons):
    start -= 1
    stop -= 1
    protein = []
    for codon_pos in range(codons):
        codon = seq[start: start + 3]
        if codon_table[codon] == "*":
            protein.append(codon_table[codon])
            break
        else:
            protein.append(codon_table[codon])
            start += 3
    return "".join(protein)
    
def find_longest_orf(orf_list):
    longest_orf = 0
    logest_orf_pos = ()
    for orf_pos in orf_list:
        orf_len = orf_pos[1] - orf_pos[0] + 1
        if orf_len > longest_orf:
            longest_orf = orf_len
            logest_orf_pos = orf_pos
    return logest_orf_pos[0], logest_orf_pos[1], int(longest_orf / 3 - 1)
    
def read_file(filepath):
    with open(filepath) as f:
        seq = []
        for line in f:
            if not line.startswith(">"):
                seq.append(line.strip())
    return "".join(seq)

def off_not_rna(comp_rna, pos_ref_rna):
    entry = ref_rna_entry.split(".")[0]
    #NM_007294.4:r.2949a>c
    notation = "{}:r.{}{}>{}".format(entry, pos_ref_rna, comp_rna[0].lower(), comp_rna[1].lower())
    return notation

def off_not_prot(comp_prot):
    entry = ref_protein_entry.split(".")[0]
    #NP_009225.1_1:p.(I946L)
    if comp_prot[0] == "silent":
        notation = "{}:p.({})".format(entry, comp_prot[0])
    else:
        notation = "{}:p.({}{}{})".format(entry, comp_prot[0], comp_prot[2], comp_prot[1])
    
    return notation

def main():
    folder = "files"
    out_file = "answers.csv"
    num_of_files = 0
    ref_cds = read_file(ref_cds_entry)
    ref_cds_prot = translate(ref_cds, 1, len(ref_cds) + 1, len(ref_cds) // 3)
    ref_rna = read_file(ref_rna_entry)
    orf_pos_ref_rna = find_longest_orf(find_orfs(ref_rna))
    ref_protein = read_file(ref_protein_entry)
    for filename in os.listdir(folder):
        if filename.endswith(".txt"):
            num_of_files += 1
    result_list = []
    for filenum in range(num_of_files):
        filepath = os.path.join("files", str(filenum + 1) + ".txt")
        #start with the RNA mapping
        seq = read_file(filepath)
        comp_rna = compare_rna(ref_cds, seq)
        pos_ref_rna = map_to_refseq(comp_rna, orf_pos_ref_rna)
        mutation_rna = off_not_rna(comp_rna, pos_ref_rna)
        # Now the protien mapping
        start = 1
        stop = len(seq) + 1
        codons = stop // 3
        aa_seq = translate(seq, start, stop, codons)
        comp_prot = compare_prot(ref_cds_prot, aa_seq)
        mutation_prot = off_not_prot(comp_prot)
        print(filepath, mutation_rna, mutation_prot)
        result_list.append((mutation_rna, mutation_prot))
    with open(out_file, "w") as f:
        for i in result_list:
            f.write("{};{}\n".format(i[0], i[1]))
    print("done")
    
main()

files\1.txt NM_007294_4:r.2949a>c NP_009225_1:p.(I946L)
files\2.txt NM_007294_4:r.1526c>g NP_009225_1:p.(silent)
files\3.txt NM_007294_4:r.5279t>g NP_009225_1:p.(silent)
files\4.txt NM_007294_4:r.2104g>a NP_009225_1:p.(R664K)
files\5.txt NM_007294_4:r.3828t>a NP_009225_1:p.(S1239T)
files\6.txt NM_007294_4:r.4144a>g NP_009225_1:p.(D1344G)
files\7.txt NM_007294_4:r.1961t>c NP_009225_1:p.(silent)
files\8.txt NM_007294_4:r.2464c>g NP_009225_1:p.(S784W)
files\9.txt NM_007294_4:r.237a>c NP_009225_1:p.(I42L)
files\10.txt NM_007294_4:r.5091g>t NP_009225_1:p.(E1660*)
files\11.txt NM_007294_4:r.4478a>c NP_009225_1:p.(L1455F)
files\12.txt NM_007294_4:r.5044a>g NP_009225_1:p.(E1644G)
files\13.txt NM_007294_4:r.3580a>c NP_009225_1:p.(D1156A)
files\14.txt NM_007294_4:r.479t>c NP_009225_1:p.(silent)
files\15.txt NM_007294_4:r.4372a>g NP_009225_1:p.(Q1420R)
files\16.txt NM_007294_4:r.3175g>a NP_009225_1:p.(S1021N)
files\17.txt NM_007294_4:r.5627g>a NP_009225_1:p.(silent)
files\18.txt NM_007294_4:r.268